# Biosimulators compatibility

## set-up

In [1]:
#!/usr/bin/env python

'''
use pymetadata module to create a minimal valid combine archive
using LEMS_NML2_Ex9_FN.sbml and LEMS_NML2_Ex9_FN.sedml
'''

import sys
sys.path.append("..")
import utils
import os
import pandas as pd
from IPython.display import display_markdown
import shutil
import yaml


cwd = os.getcwd()
print(cwd)

d1_plots_remote_dir = 'd1_plots'+ '_remote'
d1_plots_local_dir = 'd1_plots' + '_local'

sbml_filepath = 'LEMS_NML2_Ex9_FN.sbml'
sedml_filepath = 'LEMS_NML2_Ex9_FN_missing_xmlns.sedml' #xmlns:sbml missing

engines = utils.engines
types_dict = utils.types_dict


c:\Users\prins\GitHub\SBMLShowcase\SBML


In [2]:


#########################################################################################
# Run remotely
#########################################################################################

remote_output_dir = 'remote_results'

download_links_dict = dict()
for e in engines.keys():
    download_link = utils.run_biosimulators_remote(e, sedml_filepath, sbml_filepath)
    download_links_dict[e] = download_link

extract_dir_dict = dict()
for e, link in download_links_dict.items():
    extract_dir = utils.get_remote_results(e, link, remote_output_dir)
    extract_dir_dict[e] = extract_dir

results_remote = dict()
for e, extract_dir in extract_dir_dict.items():
    status = ""
    error_message = ""
    exception_type = ""

    log_yml_path = utils.find_file_in_dir('log.yml', extract_dir)[0]
    if not log_yml_path:
        status = None
        error_message = 'log.yml not found'
        continue
    with open(log_yml_path) as f:
        log_yml_dict = yaml.safe_load(f)
        if log_yml_dict['status'] == 'SUCCEEDED':
            status = 'pass'
        elif log_yml_dict['status'] == 'FAILED':
            status = 'FAIL'
            exception = log_yml_dict['exception']
            error_message = exception['message']
            exception_type = exception['type'] 
        else:
            status = None
        results_remote[e] = [status, error_message, exception_type] 

file_paths = utils.find_files(remote_output_dir, '.pdf')
utils.move_d1_files(file_paths, d1_plots_remote_dir)

# remove the remote results directory
if os.path.exists(remote_output_dir):
    shutil.rmtree(remote_output_dir)
    print('Removed ' + remote_output_dir + ' folder')

#########################################################################################
# Run locally
#########################################################################################

results_local = {}

output_folder = 'output'

for e in engines.keys():
    print('Running ' + e)
    output_dir = os.path.abspath(os.path.join(output_folder, e))
    record = utils.run_biosimulators_docker(e, sedml_filepath, sbml_filepath, output_dir=output_dir)
    results_local[e] = record

file_paths = utils.find_files(output_folder, '.pdf')
utils.move_d1_files(file_paths, d1_plots_local_dir)

# if it exists remove the output folder
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
    print('Removed ' + output_folder + ' folder')

pyNeuroML >>> DEBUG - Sim dict is: {'name': 'test', 'simulator': 'amici', 'simulatorVersion': '0.18.1', 'cpus': 1, 'memory': 8, 'maxTime': 20, 'envVars': [], 'purpose': 'academic', 'email': ''}
pyNeuroML >>> DEBUG - simulation_run_request is {"name":"test","simulator":"amici","simulatorVersion":"0.18.1","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}
pyNeuroML >>> DEBUG - data is:
{'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"amici","simulatorVersion":"0.18.1","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}')}
pyNeuroML >>> INFO - multipart encoded data is <MultipartEncoder: {'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"amici","simulatorVersion":"0.18.1","maxTime":20,"cpus":1,"memory":

Submitted LEMS_NML2_Ex9_FN_missing_xmlns.omex successfully with id: 66bb0d95f29e09b32cb6f010
View: https://api.biosimulations.org/runs/66bb0d95f29e09b32cb6f010
Downloads: https://api.biosimulations.org/results/66bb0d95f29e09b32cb6f010/download
Logs: https://api.biosimulations.org/logs/66bb0d95f29e09b32cb6f010?includeOutput=true


pyNeuroML >>> DEBUG - Sim dict is: {'name': 'test', 'simulator': 'brian2', 'simulatorVersion': '2.5.0.3', 'cpus': 1, 'memory': 8, 'maxTime': 20, 'envVars': [], 'purpose': 'academic', 'email': ''}
pyNeuroML >>> DEBUG - simulation_run_request is {"name":"test","simulator":"brian2","simulatorVersion":"2.5.0.3","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}
pyNeuroML >>> DEBUG - data is:
{'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"brian2","simulatorVersion":"2.5.0.3","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}')}
pyNeuroML >>> INFO - multipart encoded data is <MultipartEncoder: {'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"brian2","simulatorVersion":"2.5.0.3","maxTime":20,"cpus":1,"

Submitted LEMS_NML2_Ex9_FN_missing_xmlns.omex successfully with id: 66bb0d97f29e09b32cb6f013
View: https://api.biosimulations.org/runs/66bb0d97f29e09b32cb6f013
Downloads: https://api.biosimulations.org/results/66bb0d97f29e09b32cb6f013/download
Logs: https://api.biosimulations.org/logs/66bb0d97f29e09b32cb6f013?includeOutput=true


pyNeuroML >>> DEBUG - Sim dict is: {'name': 'test', 'simulator': 'bionetgen', 'simulatorVersion': '2.8.0', 'cpus': 1, 'memory': 8, 'maxTime': 20, 'envVars': [], 'purpose': 'academic', 'email': ''}
pyNeuroML >>> DEBUG - simulation_run_request is {"name":"test","simulator":"bionetgen","simulatorVersion":"2.8.0","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}
pyNeuroML >>> DEBUG - data is:
{'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"bionetgen","simulatorVersion":"2.8.0","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}')}
pyNeuroML >>> INFO - multipart encoded data is <MultipartEncoder: {'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"bionetgen","simulatorVersion":"2.8.0","maxTime":20,"cpus"

...

Submitted LEMS_NML2_Ex9_FN_missing_xmlns.omex successfully with id: 66bb0db8f29e09b32cb6f076
View: https://api.biosimulations.org/runs/66bb0db8f29e09b32cb6f076
Downloads: https://api.biosimulations.org/results/66bb0db8f29e09b32cb6f076/download
Logs: https://api.biosimulations.org/logs/66bb0db8f29e09b32cb6f076?includeOutput=true


pyNeuroML >>> DEBUG - Sim dict is: {'name': 'test', 'simulator': 'vcell', 'simulatorVersion': '7.5.0.99', 'cpus': 1, 'memory': 8, 'maxTime': 20, 'envVars': [], 'purpose': 'academic', 'email': ''}
pyNeuroML >>> DEBUG - simulation_run_request is {"name":"test","simulator":"vcell","simulatorVersion":"7.5.0.99","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}
pyNeuroML >>> DEBUG - data is:
{'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"vcell","simulatorVersion":"7.5.0.99","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}')}
pyNeuroML >>> INFO - multipart encoded data is <MultipartEncoder: {'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"vcell","simulatorVersion":"7.5.0.99","maxTime":20,"cpus":1,"

Submitted LEMS_NML2_Ex9_FN_missing_xmlns.omex successfully with id: 66bb0dbba641708179ad09fa
View: https://api.biosimulations.org/runs/66bb0dbba641708179ad09fa
Downloads: https://api.biosimulations.org/results/66bb0dbba641708179ad09fa/download
Logs: https://api.biosimulations.org/logs/66bb0dbba641708179ad09fa?includeOutput=true


pyNeuroML >>> DEBUG - Sim dict is: {'name': 'test', 'simulator': 'xpp', 'simulatorVersion': '8.0', 'cpus': 1, 'memory': 8, 'maxTime': 20, 'envVars': [], 'purpose': 'academic', 'email': ''}
pyNeuroML >>> DEBUG - simulation_run_request is {"name":"test","simulator":"xpp","simulatorVersion":"8.0","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}
pyNeuroML >>> DEBUG - data is:
{'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"xpp","simulatorVersion":"8.0","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}')}
pyNeuroML >>> INFO - multipart encoded data is <MultipartEncoder: {'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"xpp","simulatorVersion":"8.0","maxTime":20,"cpus":1,"memory":8,"purpose":"academi

Submitted LEMS_NML2_Ex9_FN_missing_xmlns.omex successfully with id: 66bb0dbd0c4ccdd491921074
View: https://api.biosimulations.org/runs/66bb0dbd0c4ccdd491921074
Downloads: https://api.biosimulations.org/results/66bb0dbd0c4ccdd491921074/download
Logs: https://api.biosimulations.org/logs/66bb0dbd0c4ccdd491921074?includeOutput=true
Moving remote_results\amici\outputs\tmp835455\d1.pdf to d1_plots_remote\amici_d1.pdf
Moving remote_results\bionetgen\outputs\tmp671390\d1.pdf to d1_plots_remote\bionetgen_d1.pdf
Moving remote_results\boolnet\outputs\tmp657556\d1.pdf to d1_plots_remote\boolnet_d1.pdf
Moving remote_results\cbmpy\outputs\tmp988681\d1.pdf to d1_plots_remote\cbmpy_d1.pdf
Moving remote_results\cobrapy\outputs\tmp108287\d1.pdf to d1_plots_remote\cobrapy_d1.pdf
Moving remote_results\copasi\outputs\tmp548282\d1.pdf to d1_plots_remote\copasi_d1.pdf
Moving remote_results\gillespy2\outputs\tmp739285\d1.pdf to d1_plots_remote\gillespy2_d1.pdf
Moving remote_results\ginsim\outputs\tmp535823\d1

In [5]:
#########################################################################################
# process results and save markdown table
#########################################################################################

results_table = utils.create_results_table(results_remote, types_dict, sbml_filepath, sedml_filepath, engines, d1_plots_remote_dir)
results_table_local = utils.create_results_table(results_local, types_dict, sbml_filepath, sedml_filepath, engines, d1_plots_local_dir)

# rename cols to distinguish between local and remote results except for Engine column
results_table.columns = [str(col) + ' (remote)' if col != 'Engine' else str(col) for col in results_table.columns]
results_table_local.columns = [str(col) + ' (local)' if col != 'Engine' else str(col) for col in results_table_local.columns]

# combine remote and local results
combined_results = pd.merge(results_table, results_table_local, on='Engine', how='outer')
combined_results = combined_results.reindex(columns=['Engine'] + sorted(combined_results.columns[1:]))

cols_order = ['Engine', 'pass / FAIL (remote)', 'pass / FAIL (local)',\
               'Compat (remote)', 'Compat (local)', \
               'Type (remote)', \
               'Error (remote)', 'Error (local)', \
               'd1 (remote)', 'd1 (local)']

combined_results = combined_results[cols_order]
display_markdown(combined_results.to_markdown(), raw=True)

|    | Engine     | pass / FAIL (remote)                                                                                                                                                       | pass / FAIL (local)                                                                                                                                                        | Compat (remote)                                                                                                                                                                                                                                                                                                                        | Compat (local)                                                                                                                                                                                                                                                                                                                         | Type (remote)                                                          | Error (remote)                                                                                                                                                                                                                                                                                                                                                                                                                                                     | Error (local)                                                                                                                                               | d1 (remote)                                          | d1 (local)                                          |
|---:|:-----------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------|:----------------------------------------------------|
|  0 | amici      | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with amici</details>                                                                                                                        | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with amici</details>                                                                                                                        |                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\amici_d1.pdf">plot</a>      | <a href="d1_plots_local\amici_d1.pdf">plot</a>      |
|  1 | bionetgen  | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with bionetgen. The following file types will be compatible ['BNGL', 'SED-ML']</details></span>            | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with bionetgen. The following file types will be compatible ['BNGL', 'SED-ML']</details></span>            | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Language for model `net1` is not supported.<br>      - Model language `urn:sedml:language:sbml` is not supported. Models must be in BNGL format (e.g., `sed:model/@language` must match `^urn:sedml:language:bngl(\.$)` such as `urn:sedml:language:bngl`).</details>              | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\bionetgen_d1.pdf">plot</a>  | <a href="d1_plots_local\bionetgen_d1.pdf">plot</a>  |
|  2 | boolnet    | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with boolnet. The following file types will be compatible ['SBML-qual', 'SED-ML']</details></span>         | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with boolnet. The following file types will be compatible ['SBML-qual', 'SED-ML']</details></span>         | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Simulation `sim1` is invalid.<br>      - Number of points (20000) must be equal to the difference between the output end (200.0) and start times (0.0).</details>                                                                                                                  | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\boolnet_d1.pdf">plot</a>    | <a href="d1_plots_local\boolnet_d1.pdf">plot</a>    |
|  3 | brian2     | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with brian2</details>                                                                                                                       | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with brian2</details>                                                                                                                       | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> |                                                      |                                                     |
|  4 | cbmpy      | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with cbmpy</details>                                                                                                                        | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with cbmpy</details>                                                                                                                        | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    UniformTimeCourseSimulation `sim1` is not supported.<br>      - Simulation sim1 of type `UniformTimeCourseSimulation` is not supported. Simulation must be an instance of one of the following:<br>          - SteadyStateSimulation</details>                                     | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\cbmpy_d1.pdf">plot</a>      | <a href="d1_plots_local\cbmpy_d1.pdf">plot</a>      |
|  5 | cobrapy    | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with cobrapy</details>                                                                                                                      | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with cobrapy</details>                                                                                                                      | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    UniformTimeCourseSimulation `sim1` is not supported.<br>      - Simulation sim1 of type `UniformTimeCourseSimulation` is not supported. Simulation must be an instance of one of the following:<br>          - SteadyStateSimulation</details>                                     | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\cobrapy_d1.pdf">plot</a>    | <a href="d1_plots_local\cobrapy_d1.pdf">plot</a>    |
|  6 | copasi     | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with copasi</details>                                                                                                                       | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with copasi</details>                                                                                                                       | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    could not convert string to float: 'Values[V]'</details>                                                                                                                                                                                                                           | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\copasi_d1.pdf">plot</a>     | <a href="d1_plots_local\copasi_d1.pdf">plot</a>     |
|  7 | gillespy2  | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with gillespy2</details>                                                                                                                    | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with gillespy2</details>                                                                                                                    |                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\gillespy2_d1.pdf">plot</a>  | <a href="d1_plots_local\gillespy2_d1.pdf">plot</a>  |
|  8 | ginsim     | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with ginsim. The following file types will be compatible ['SBML-qual', 'SED-ML']</details></span>          | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with ginsim. The following file types will be compatible ['SBML-qual', 'SED-ML']</details></span>          | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Simulation `sim1` is invalid.<br>      - The interval between the output start and time time must be an integer multiple of the number of steps, not `0.01`:<br>          Output start time: 0.0<br>          Output end time: 200.0<br>          Number of steps: 20000</details> | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\ginsim_d1.pdf">plot</a>     | <a href="d1_plots_local\ginsim_d1.pdf">plot</a>     |
|  9 | libsbmlsim | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with libsbmlsim</details>                                                                                                                   | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with libsbmlsim</details>                                                                                                                   |                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\libsbmlsim_d1.pdf">plot</a> | <a href="d1_plots_local\libsbmlsim_d1.pdf">plot</a> |
| 10 | masspy     | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with masspy</details>                                                                                                                       | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with masspy</details>                                                                                                                       | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Something went wrong reading the SBML model. Most likely the SBML model is not valid. Please check that your model is valid using the `mass.io.sbml.validate_sbml_model` function or via the online validator at http://sbml.org/validator .<br>    	`(model, errors) = validate_sbml_model(filename)`<br>    If the model is valid and cannot be read please open an issue at https://github.com/SBRG/masspy/issues .</details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\masspy_d1.pdf">plot</a>     | <a href="d1_plots_local\masspy_d1.pdf">plot</a>     |
| 11 | netpyne    | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with netpyne</details>                                                                                                                      | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with netpyne</details>                                                                                                                      | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> |                                                      |                                                     |
| 12 | neuron     | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with neuron. The following file types will be compatible ['NeuroML', 'LEMS', 'SED-ML']</details></span>    | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with neuron. The following file types will be compatible ['NeuroML', 'LEMS', 'SED-ML']</details></span>    | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> |                                                      |                                                     |
| 13 | opencor    | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with opencor. The following file types will be compatible ['CellML', 'SED-ML']</details></span>            | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with opencor. The following file types will be compatible ['CellML', 'SED-ML']</details></span>            | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> |                                                      |                                                     |
| 14 | pyneuroml  | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with pyneuroml. The following file types will be compatible ['NeuroML', 'LEMS', 'SED-ML']</details></span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with pyneuroml. The following file types will be compatible ['NeuroML', 'LEMS', 'SED-ML']</details></span> | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> |                                                      |                                                     |
| 15 | pysces     | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with pysces</details>                                                                                                                       | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with pysces</details>                                                                                                                       | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Model at /tmp/tmp8ak6qa9f/./LEMS_NML2_Ex9_FN.sbml could not be imported:<br>      <br>      File /tmp/tmp8ak6qa9f/./LEMS_NML2_Ex9_FN.sbml.xml does not exist</details>                                                                                                             | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\pysces_d1.pdf">plot</a>     | <a href="d1_plots_local\pysces_d1.pdf">plot</a>     |
| 16 | rbapy      | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with rbapy. The following file types will be compatible ['RBApy', 'SED-ML']</details></span>               | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with rbapy. The following file types will be compatible ['RBApy', 'SED-ML']</details></span>               | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Language for model `net1` is not supported.<br>      - Model language `urn:sedml:language:sbml` is not supported. Models must be in RBA format (e.g., `sed:model/@language` must match `^urn:sedml:language:rba(\.$)` such as `urn:sedml:language:rba`).</details>                 | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\rbapy_d1.pdf">plot</a>      | <a href="d1_plots_local\rbapy_d1.pdf">plot</a>      |
| 17 | smoldyn    | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>smoldyn compatible file types unknown.</details></span>                                                                                                                                                       | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>smoldyn compatible file types unknown.</details></span>                                                                                                                                                       | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> |                                                      |                                                     |
| 18 | tellurium  | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with tellurium</details>                                                                                                                    | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with tellurium</details>                                                                                                                    |                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> | <a href="d1_plots_remote\tellurium_d1.pdf">plot</a>  | <a href="d1_plots_local\tellurium_d1.pdf">plot</a>  |
| 19 | vcell      | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>vcell compatible file types unknown.</details></span>                                                                                                                                                         | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>vcell compatible file types unknown.</details></span>                                                                                                                                                         |                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> |                                                      |                                                     |
| 20 | xpp        | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with xpp. The following file types will be compatible ['XPP', 'SED-ML']</details></span>                   | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. Tese are not compatible with xpp. The following file types will be compatible ['XPP', 'SED-ML']</details></span>                   | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')```</details> |                                                      |                                                     |